In [1]:
from Get_Data import get_historical_future_data,get_historical_text_data,get_recent_historical_data
from Prompt import get_expert_prediction,get_prediction_from_historical
import numpy as np  
from Get_Data import get_historical_future_data, get_historical_text_data
from Tool import (
    format_dict_to_string,
    format_historical_data,
    calculate_metrics,
    visualize_predictions,
    save_experiment_results_with_repeats,
    calculate_metrics_statistics
)
from Prompt import (
    get_expert_prediction,
    get_unimodal_prediction,
    get_multimodal_prediction
)
import json
import os
from datetime import datetime
def generate_latex_table_from_results(save_dir="experiment_results", targets=None, months=None, llm_family="GPT4", Multi=False):
    """
    从保存的实验结果生成LaTeX表格
    
    参数:
    save_dir (str): 结果文件所在目录
    targets (list): 目标名称列表，如果为None则处理所有发现的目标
    months (list): 月份列表，如果为None则处理所有发现的月份
    llm_family (str): 语言模型系列名称
    Multi (bool): 是否是多模态结果
    
    返回:
    str: LaTeX表格字符串
    """
    # 读取所有结果文件
    results = {}
    suffix = "Multi" if Multi else "Uni"
    
    # 如果没有指定targets和months，先扫描目录获取所有可能的值
    if targets is None or months is None:
        targets = set()
        months = set()
        for filename in os.listdir(save_dir):
            if filename.endswith(f"{llm_family}_{suffix}.json"):
                parts = filename.split('_')
                target = parts[0].replace('_', ' ')
                month = int(parts[1].replace('m', ''))
                targets.add(target)
                months.add(month)
        targets = sorted(list(targets))
        months = sorted(list(months))
    
    for target in targets:
        results[target] = {}
        for month in months:
            filename = f"{target.replace(' ', '_')}_{month}m_{llm_family}_{suffix}.json"
            filepath = os.path.join(save_dir, filename)
            if os.path.exists(filepath):
                try:
                    with open(filepath, 'r') as f:
                        data = json.load(f)
                    # 检查数据结构并获取正确的metrics
                    if 'all_metrics' in data:
                        results[target][month] = data
                    else:
                        print(f"警告: {filename} 中没有找到 all_metrics")
                        continue
                except Exception as e:
                    print(f"读取文件 {filename} 时出错: {str(e)}")
                    continue
    
    if not results:
        print("没有找到任何有效的结果文件")
        return ""
    
    # 获取第一个有效结果来确定方法列表
    first_target = next(iter(results))
    first_month = next(iter(results[first_target]))
    first_data = results[first_target][first_month]
    
    if 'all_metrics' not in first_data:
        print("错误: 找不到 all_metrics 数据")
        return ""
        
    # 获取第一次重复实验的结果
    first_repeat = next(iter(first_data['all_metrics'].values()))
    
    # 获取方法列表并简化名称
    methods = list(first_repeat.keys())
    # 只保留当前模态的方法
    prefix = "Multimodal_" if Multi else "Unimodal_"
    methods = [m for m in methods if m.startswith(prefix)]
    # 简化方法名称
    simplified_methods = [m.replace(prefix, "") for m in methods]
    
    # 初始化win_counts字典
    win_counts = {method: 0 for method in methods}
    # 修改表头部分
    latex = "\\begin{table*}[htbp]\n\\centering\n"  # 改为table*
    latex += "\\newcommand{\\better}{\\textcolor{green}}\n"
    latex += "\\newcommand{\\worse}{\\textcolor{red}}\n"
    latex += "\\begin{tabular}{l|"
    latex += "".join(["c" for _ in simplified_methods]) + "}\n\\hline\n"
    
    # 添加表头 - 处理特殊方法名称
    latex += "Target"  # 删除 Month &
    for method in simplified_methods:
        # 处理特殊方法名称
        if method == "self_consistency":
            method_display = "Self Consistency"
        elif method == "self_correction":
            method_display = "Self Correction"
        else:
            method_display = method
        latex += f" & {method_display}"
    latex += " \\\\\n\\hline\n"
    
    # 添加数据行
    for month in months:
        for target in targets:
            if month in results[target]:
                stats = calculate_metrics_statistics(results[target][month]['all_metrics'])
                
                latex += target  # 删除month相关的判断逻辑
                
                # 获取naive方法的MSE
                naive_method = f"{prefix}naive"
                naive_mse = stats[naive_method]['nmse_mean'] if naive_method in stats else float('inf')
                
                for method in methods:
                    if method in stats:
                        mse = stats[method]['nmse_mean']
                        std = stats[method]['nmse_std']
                        
                        # 格式化数值显示
                        if method != naive_method:
                            if mse < naive_mse:
                                latex += f" & \\better{{{mse:.3f}$\\pm${std:.3f}}}"
                                win_counts[method] += 1
                            else:
                                latex += f" & \\worse{{{mse:.3f}$\\pm${std:.3f}}}"
                        else:
                            latex += f" & {mse:.3f}$\\pm${std:.3f}"
                    else:
                        latex += " & -"
                latex += " \\\\\n"

    # 修改Win/Naive统计行 - 使用分数表示
    latex += "\\hline\n"
    latex += "Win/Naive"
    for method in methods:
        if method != naive_method:
            wins = win_counts[method]
            total = len(targets)
            latex += f" & ${wins}/{total}$"
        else:
            latex += " & -"
    latex += " \\\\\n"

    # 修改表格尾部
    latex += "\\hline\n\\end{tabular}\n"
    latex += f"\\caption{{Performance comparison of different {prefix.rstrip('_')} methods ({llm_family}))}}\n"
    latex += f"\\label{{tab:performance_comparison_{prefix.lower().rstrip('_')}}}\n"
    latex += "\\end{table*}"

    return latex

In [15]:
target_list=[
"the US Retail Broiler Composit",
"the US Drought Level",
"the US International Trade Balance",
"the US Gasoline Prices",
"the US Influenza Patients Proportion",
"the US Disaster and Emergency Grant",
"the US Unemployment Rate",
"the US Travel Volume"
]
# 生成单模态表格
unimodal_table = generate_latex_table_from_results(
    save_dir="experiment_results",
    targets=target_list,
    months=[6],
    llm_family="DeepSeek",
    Multi=0
)

# 生成多模态表格
multimodal_table = generate_latex_table_from_results(
    save_dir="experiment_results",
    targets=target_list,
    months=[6],
    llm_family="DeepSeek",
    Multi=1
)

In [16]:
print(unimodal_table)

\begin{table*}[htbp]
\centering
\newcommand{\better}{\textcolor{green}}
\newcommand{\worse}{\textcolor{red}}
\begin{tabular}{l|ccccc}
\hline
Target & naive & cot & Self Consistency & Self Correction & lrm \\
\hline
the US Retail Broiler Composit & 0.216$\pm$0.049 & \better{0.102$\pm$0.034} & \better{0.103$\pm$0.014} & \better{0.121$\pm$0.065} & \better{0.091$\pm$0.019} \\
the US Drought Level & 0.902$\pm$0.000 & \worse{1.383$\pm$0.227} & \better{0.786$\pm$0.153} & \worse{0.913$\pm$0.078} & \better{0.662$\pm$0.051} \\
the US International Trade Balance & 0.613$\pm$0.776 & \better{0.540$\pm$0.386} & \better{0.393$\pm$0.113} & \worse{0.948$\pm$0.589} & \better{0.359$\pm$0.000} \\
the US Gasoline Prices & 0.603$\pm$0.359 & \better{0.575$\pm$0.452} & \worse{0.923$\pm$0.265} & \better{0.332$\pm$0.150} & \worse{1.396$\pm$0.000} \\
the US Influenza Patients Proportion & 0.841$\pm$0.215 & \better{0.658$\pm$0.227} & \better{0.538$\pm$0.021} & \worse{0.939$\pm$0.328} & \worse{0.972$\pm$0.533} \\


In [17]:
print(multimodal_table)

\begin{table*}[htbp]
\centering
\newcommand{\better}{\textcolor{green}}
\newcommand{\worse}{\textcolor{red}}
\begin{tabular}{l|ccccc}
\hline
Target & naive & cot & Self Consistency & Self Correction & lrm \\
\hline
the US Retail Broiler Composit & 0.088$\pm$0.058 & \better{0.063$\pm$0.022} & \worse{0.136$\pm$0.080} & \worse{0.119$\pm$0.078} & \better{0.019$\pm$0.010} \\
the US Drought Level & 0.897$\pm$0.000 & \worse{2.193$\pm$0.330} & \worse{0.897$\pm$0.000} & \worse{0.939$\pm$0.074} & \worse{1.849$\pm$0.570} \\
the US International Trade Balance & 0.629$\pm$0.147 & \better{0.558$\pm$0.282} & \better{0.486$\pm$0.074} & \better{0.623$\pm$0.218} & \worse{0.806$\pm$0.354} \\
the US Gasoline Prices & 0.995$\pm$0.139 & \worse{1.286$\pm$0.568} & \better{0.809$\pm$0.241} & \better{0.493$\pm$0.112} & \better{0.746$\pm$0.459} \\
the US Influenza Patients Proportion & 2.624$\pm$2.400 & \better{0.974$\pm$0.446} & \better{0.644$\pm$0.488} & \better{1.135$\pm$0.643} & \better{1.560$\pm$0.957} \\
t